# Task description
- Classify the speakers of given features.
- Main goal: Learn how to use transformer.
- Baselines:
  - Easy: Run sample code and know how to use transformer.
  - Medium: Know how to adjust parameters of transformer.
  - Strong: Construct [conformer](https://arxiv.org/abs/2005.08100) which is a variety of transformer.
  - Boss: Implement [Self-Attention Pooling](https://arxiv.org/pdf/2008.01077v1.pdf) & [Additive Margin Softmax](https://arxiv.org/pdf/1801.05599.pdf) to further boost the performance.

- Other links
  - Competiton: [link](https://www.kaggle.com/t/49ea0c385a974db5919ec67299ba2e6b)
  - Slide: [link](https://docs.google.com/presentation/d/1LDAW0GGrC9B6D7dlNdYzQL6D60-iKgFr/edit?usp=sharing&ouid=104280564485377739218&rtpof=true&sd=true)
  - Data: [link](https://github.com/googly-mingto/ML2023HW4/releases)



In [1]:
import numpy as np
import torch
import random

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(42)

# Data

## Dataset
- Original dataset is [Voxceleb2](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox2.html).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb2.
- We randomly select 600 speakers from Voxceleb2.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training.
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary.
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [2]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence


class myDataset(Dataset):
	def __init__(self, data_dir, segment_len=128):
		self.data_dir = data_dir
		self.segment_len = segment_len

		# Load the mapping from speaker neme to their corresponding id.
		mapping_path = Path(data_dir) / "mapping.json"
		mapping = json.load(mapping_path.open())
		self.speaker2id = mapping["speaker2id"]

		# Load metadata of training data.
		metadata_path = Path(data_dir) / "metadata.json"
		metadata = json.load(open(metadata_path))["speakers"]

		# Get the total number of speaker.
		self.speaker_num = len(metadata.keys())
		self.data = []
		for speaker in metadata.keys():
			for utterances in metadata[speaker]:
				self.data.append([utterances["feature_path"], self.speaker2id[speaker]])

	def __len__(self):
			return len(self.data)

	def __getitem__(self, index):
		feat_path, speaker = self.data[index]
		# Load preprocessed mel-spectrogram.
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		# Segmemt mel-spectrogram into "segment_len" frames.
		if len(mel) > self.segment_len:
			# Randomly get the starting point of the segment.
			start = random.randint(0, len(mel) - self.segment_len)
			# Get a segment with "segment_len" frames.
			mel = torch.FloatTensor(mel[start:start+self.segment_len])
		else:
			mel = torch.FloatTensor(mel)
		# Turn the speaker id into long for computing loss later.
		speaker = torch.FloatTensor([speaker]).long()
		return mel, speaker

	def get_speaker_number(self):
		return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.

In [3]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
	# Process features within a batch.
	"""Collate a batch of data."""
	mel, speaker = zip(*batch)
	# Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
	mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
	# mel: (batch size, length, 40)
	return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
	"""Generate dataloader"""
	dataset = myDataset(data_dir)
	speaker_num = dataset.get_speaker_number()
	# Split dataset into training dataset and validation dataset
	trainlen = int(0.9 * len(dataset))
	lengths = [trainlen, len(dataset) - trainlen]
	trainset, validset = random_split(dataset, lengths)

	train_loader = DataLoader(
		trainset,
		batch_size=batch_size,
		shuffle=True,
		drop_last=True,
		num_workers=n_workers,
		pin_memory=True,
		collate_fn=collate_batch,
	)
	valid_loader = DataLoader(
		validset,
		batch_size=batch_size,
		num_workers=n_workers,
		drop_last=True,
		pin_memory=True,
		collate_fn=collate_batch,
	)

	return train_loader, valid_loader, speaker_num

# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

"""Conformer"""
class ConformerLayer(nn.Module):
    def __init__(self,d_model, dim_feedforward, n_head) -> None:
        super(ConformerLayer,self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
			d_model=d_model, dim_feedforward=dim_feedforward, nhead=n_head
		)
        self.conv1d = nn.Conv1d(d_model,d_model,kernel_size=3,padding=1)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(dim_feedforward,d_model)
        self.norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.encoder_layer(x)
        output = x.transpose(1,2)
        output = self.conv1d(output)
        output = output.transpose(1,2)
        x = x + self.dropout(output)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.norm(x)

        return x
    
class ConformerEncoder(nn.Module):
    def __init__(self,d_model, dim_feedforward, n_head):
        super(ConformerEncoder,self).__init__()
        self.layer = ConformerLayer(d_model,dim_feedforward,n_head)
    
    def forward(self,x):
        x = self.layer(x)
        return x
    
class SelfAttentionPooling(nn.Module):
    def __init__(self, d_model):
        super(SelfAttentionPooling, self).__init__()
        self.W = nn.Linear(d_model, 1)

    def forward(self, x):
        attn_weights = F.softmax(self.W(x), dim=1)
        pooled_output = torch.sum(attn_weights * x, dim=1)
        return pooled_output

class Classifier(nn.Module):
    def __init__(self, d_model=120, n_spks=600, dropout=0.2):
        super(Classifier, self).__init__()
        self.prenet = nn.Linear(40, d_model)
        # self.encoder_layer = nn.TransformerEncoderLayer(
		# 	d_model=d_model, dim_feedforward=256, nhead=4
		# )
        # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=4)
        self.encoder = ConformerEncoder(d_model,dim_feedforward=256,n_head=8)
        self.self_attention_pooling = SelfAttentionPooling(d_model)
        self.pred_layer = nn.Sequential(
            nn.Linear(d_model, 240),
            nn.Linear(240, 480),
            nn.Sigmoid(),
            nn.Linear(480, n_spks),
        )

    def forward(self, mels):
        out = self.prenet(mels)
        out = out.permute(1, 0, 2)
        # out = self.self_attention_pooling(out)
        out = self.encoder(out)
		# out: (batch size, length, d_model)
        out = out.transpose(0, 1)
		# mean pooling
        # stats = out.mean(dim=1)
        stats = self.self_attention_pooling(out)

		# out: (batch, n_spks)
        out = self.pred_layer(stats)
        return out

# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [5]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
	optimizer: Optimizer,
	num_warmup_steps: int,
	num_training_steps: int,
	num_cycles: float = 0.5,
	last_epoch: int = -1,
):
	"""
	Create a schedule with a learning rate that decreases following the values of the cosine function between the
	initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
	initial lr set in the optimizer.

	Args:
		optimizer (:class:`~torch.optim.Optimizer`):
		The optimizer for which to schedule the learning rate.
		num_warmup_steps (:obj:`int`):
		The number of steps for the warmup phase.
		num_training_steps (:obj:`int`):
		The total number of training steps.
		num_cycles (:obj:`float`, `optional`, defaults to 0.5):
		The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
		following a half-cosine).
		last_epoch (:obj:`int`, `optional`, defaults to -1):
		The index of the last epoch when resuming training.

	Return:
		:obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
	"""
	def lr_lambda(current_step):
		# Warmup
		if current_step < num_warmup_steps:
			return float(current_step) / float(max(1, num_warmup_steps))
		# decadence
		progress = float(current_step - num_warmup_steps) / float(
			max(1, num_training_steps - num_warmup_steps)
		)
		return max(
			0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
		)

	return LambdaLR(optimizer, lr_lambda, last_epoch)

# Model Function
- Model forward function.

In [6]:
import torch


def model_fn(batch, model, criterion, device):
	"""Forward a batch through the model."""

	mels, labels = batch
	mels = mels.to(device)
	labels = labels.to(device)

	outs = model(mels)

	loss = criterion(outs, labels)

	# Get the speaker id with highest probability.
	preds = outs.argmax(1)
	# Compute accuracy.
	accuracy = torch.mean((preds == labels).float())

	return loss, accuracy

# Validate
- Calculate accuracy of the validation set.

In [7]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device):
	"""Validate on validation set."""

	model.eval()
	running_loss = 0.0
	running_accuracy = 0.0
	pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

	for i, batch in enumerate(dataloader):
		with torch.no_grad():
			loss, accuracy = model_fn(batch, model, criterion, device)
			running_loss += loss.item()
			running_accuracy += accuracy.item()

		pbar.update(dataloader.batch_size)
		pbar.set_postfix(
			loss=f"{running_loss / (i+1):.2f}",
			accuracy=f"{running_accuracy / (i+1):.2f}",
		)

	pbar.close()
	model.train()

	return running_accuracy / len(dataloader)

# Main function

In [8]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset",
		"save_path": "model.ckpt",
		"batch_size": 128,
		"n_workers": 8,
		"valid_steps": 2000,
		"warmup_steps": 1000,
		"save_steps": 10000,
		"total_steps": 70000,
	}

	return config


def main(
	data_dir,
	save_path,
	batch_size,
	n_workers,
	valid_steps,
	warmup_steps,
	total_steps,
	save_steps,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
	train_iterator = iter(train_loader)
	print(f"[Info]: Finish loading data!",flush = True)

	model = Classifier(n_spks=speaker_num).to(device)
	criterion = nn.CrossEntropyLoss()
	optimizer = AdamW(model.parameters(), lr=1e-3)
	scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
	print(f"[Info]: Finish creating model!",flush = True)

	best_accuracy = -1.0
	best_state_dict = None

	pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

	for step in range(total_steps):
		# Get data
		try:
			batch = next(train_iterator)
		except StopIteration:
			train_iterator = iter(train_loader)
			batch = next(train_iterator)

		loss, accuracy = model_fn(batch, model, criterion, device)
		batch_loss = loss.item()
		batch_accuracy = accuracy.item()

		# Updata model
		loss.backward()
		optimizer.step()
		scheduler.step()
		optimizer.zero_grad()

		# Log
		pbar.update()
		pbar.set_postfix(
			loss=f"{batch_loss:.2f}",
			accuracy=f"{batch_accuracy:.2f}",
			step=step + 1,
		)

		# Do validation
		if (step + 1) % valid_steps == 0:
			pbar.close()

			valid_accuracy = valid(valid_loader, model, criterion, device)

			# keep the best model
			if valid_accuracy > best_accuracy:
				best_accuracy = valid_accuracy
				best_state_dict = model.state_dict()

			pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

		# Save the best model so far.
		if (step + 1) % save_steps == 0 and best_state_dict is not None:
			torch.save(best_state_dict, save_path)
			pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

	pbar.close()


if __name__ == "__main__":
	main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [01:10<00:00, 28.42 step/s, accuracy=0.15, loss=4.12, step=2000]
Valid:  99% 5632/5667 [00:00<00:00, 5728.82 uttr/s, accuracy=0.14, loss=4.03]
Train: 100% 2000/2000 [01:14<00:00, 27.02 step/s, accuracy=0.20, loss=3.51, step=4000]
Valid:  99% 5632/5667 [00:00<00:00, 6075.23 uttr/s, accuracy=0.20, loss=3.65]
Train: 100% 2000/2000 [01:13<00:00, 27.07 step/s, accuracy=0.23, loss=3.13, step=6000]
Valid:  99% 5632/5667 [00:00<00:00, 6102.12 uttr/s, accuracy=0.27, loss=3.27]
Train: 100% 2000/2000 [01:13<00:00, 27.10 step/s, accuracy=0.30, loss=2.91, step=8000]
Valid:  99% 5632/5667 [00:00<00:00, 5789.36 uttr/s, accuracy=0.31, loss=3.08]
Train: 100% 2000/2000 [01:13<00:00, 27.07 step/s, accuracy=0.47, loss=2.52, step=1e+4]
Valid:  99% 5632/5667 [00:00<00:00, 5814.44 uttr/s, accuracy=0.34, loss=2.93]
Train:   0% 5/2000 [00:00<01:19, 24.96 step/s, accuracy=0.30, loss=2.64, step=1e+4]

Step 10000, best model saved. (accuracy=0.3382)


Train: 100% 2000/2000 [01:14<00:00, 26.92 step/s, accuracy=0.38, loss=2.48, step=12000]
Valid:  99% 5632/5667 [00:01<00:00, 5592.09 uttr/s, accuracy=0.37, loss=2.74]
Train: 100% 2000/2000 [01:14<00:00, 26.98 step/s, accuracy=0.40, loss=2.64, step=14000]
Valid:  99% 5632/5667 [00:00<00:00, 5632.42 uttr/s, accuracy=0.39, loss=2.66]
Train: 100% 2000/2000 [01:14<00:00, 26.94 step/s, accuracy=0.52, loss=2.29, step=16000]
Valid:  99% 5632/5667 [00:00<00:00, 6028.48 uttr/s, accuracy=0.42, loss=2.54]
Train: 100% 2000/2000 [01:14<00:00, 26.99 step/s, accuracy=0.42, loss=2.32, step=18000]
Valid:  99% 5632/5667 [00:00<00:00, 5903.10 uttr/s, accuracy=0.43, loss=2.44]
Train: 100% 2000/2000 [01:14<00:00, 27.01 step/s, accuracy=0.50, loss=2.12, step=2e+4] 
Valid:  99% 5632/5667 [00:00<00:00, 6088.02 uttr/s, accuracy=0.44, loss=2.43]
Train:   0% 5/2000 [00:00<01:11, 27.93 step/s, accuracy=0.52, loss=2.03, step=2e+4]

Step 20000, best model saved. (accuracy=0.4439)


...

Step 30000, best model saved. (accuracy=0.5192)


Train: 100% 2000/2000 [01:13<00:00, 27.04 step/s, accuracy=0.61, loss=1.45, step=32000]
Valid:  99% 5632/5667 [00:01<00:00, 5477.55 uttr/s, accuracy=0.55, loss=1.93]
Train: 100% 2000/2000 [01:13<00:00, 27.13 step/s, accuracy=0.58, loss=1.55, step=34000]
Valid:  99% 5632/5667 [00:01<00:00, 5172.37 uttr/s, accuracy=0.55, loss=1.94]
Train: 100% 2000/2000 [01:14<00:00, 27.01 step/s, accuracy=0.63, loss=1.37, step=36000]
Valid:  99% 5632/5667 [00:01<00:00, 5489.55 uttr/s, accuracy=0.56, loss=1.90]
Train: 100% 2000/2000 [01:14<00:00, 26.94 step/s, accuracy=0.69, loss=1.28, step=38000]
Valid:  99% 5632/5667 [00:00<00:00, 6130.76 uttr/s, accuracy=0.57, loss=1.90]
Train: 100% 2000/2000 [01:18<00:00, 25.33 step/s, accuracy=0.70, loss=1.11, step=4e+4] 
Valid:  99% 5632/5667 [00:01<00:00, 5594.76 uttr/s, accuracy=0.57, loss=1.87]
Train:   0% 5/2000 [00:00<01:24, 23.66 step/s, accuracy=0.66, loss=1.39, step=4e+4]

Step 40000, best model saved. (accuracy=0.5716)


Train: 100% 2000/2000 [01:14<00:00, 26.90 step/s, accuracy=0.70, loss=1.18, step=42000]
Valid:  99% 5632/5667 [00:00<00:00, 6226.72 uttr/s, accuracy=0.58, loss=1.83]
Train: 100% 2000/2000 [01:14<00:00, 26.97 step/s, accuracy=0.73, loss=0.96, step=44000]
Valid:  99% 5632/5667 [00:00<00:00, 6086.56 uttr/s, accuracy=0.59, loss=1.76]
Train: 100% 2000/2000 [01:14<00:00, 26.93 step/s, accuracy=0.80, loss=0.85, step=46000]
Valid:  99% 5632/5667 [00:00<00:00, 6089.76 uttr/s, accuracy=0.60, loss=1.75]
Train:   8% 150/2000 [00:05<01:06, 27.97 step/s, accuracy=0.62, loss=1.24, step=46150]

KeyboardInterrupt: 

# Inference

## Dataset of inference

In [ ]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
	def __init__(self, data_dir):
		testdata_path = Path(data_dir) / "testdata.json"
		metadata = json.load(testdata_path.open())
		self.data_dir = data_dir
		self.data = metadata["utterances"]

	def __len__(self):
		return len(self.data)

	def __getitem__(self, index):
		utterance = self.data[index]
		feat_path = utterance["feature_path"]
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		return feat_path, mel


def inference_collate_batch(batch):
	"""Collate a batch of data."""
	feat_paths, mels = zip(*batch)

	return feat_paths, torch.stack(mels)

## Main funcrion of Inference

In [ ]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset",
		"model_path": "./model.ckpt",
		"output_path": "./output.csv",
	}

	return config


def main(
	data_dir,
	model_path,
	output_path,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	mapping_path = Path(data_dir) / "mapping.json"
	mapping = json.load(mapping_path.open())

	dataset = InferenceDataset(data_dir)
	dataloader = DataLoader(
		dataset,
		batch_size=1,
		shuffle=False,
		drop_last=False,
		num_workers=8,
		collate_fn=inference_collate_batch,
	)
	print(f"[Info]: Finish loading data!",flush = True)

	speaker_num = len(mapping["id2speaker"])
	model = Classifier(n_spks=speaker_num).to(device)
	model.load_state_dict(torch.load(model_path))
	model.eval()
	print(f"[Info]: Finish creating model!",flush = True)

	results = [["Id", "Category"]]
	for feat_paths, mels in tqdm(dataloader):
		with torch.no_grad():
			mels = mels.to(device)
			outs = model(mels)
			preds = outs.argmax(1).cpu().numpy()
			for feat_path, pred in zip(feat_paths, preds):
				results.append([feat_path, mapping["id2speaker"][str(pred)]])

	with open(output_path, 'w', newline='') as csvfile:
		writer = csv.writer(csvfile)
		writer.writerows(results)


if __name__ == "__main__":
	main(**parse_args())